In [ ]:
# coding=utf-8

import os
import struct
import numpy as np
import tensorflow as tf
from read_mnist import load_mnist
from sklearn.preprocessing import StandardScaler

# 生成batch
def batch_iter(data,batchsize=500):
    datasize = len(data)
    num_batch = int(datasize/batchsize)
    for itr in range(num_batch):
        bg = itr*batchsize
        ed = min((itr+1)*batchsize,datasize)
        yield data[bg:ed]

    
    
# ---------------------------加载数据---------------------------------
train_x, train_y = load_mnist("mnist",kind='train')
test_x, test_y = load_mnist("mnist",kind="t10k")

train_x = StandardScaler().fit_transform(train_x)
test_x = StandardScaler().fit_transform(test_x)

train_y_onehot = np.zeros([train_y.shape[0],10])
train_y_onehot[range(train_y.shape[0]),train_y]=1
test_y_onehot = np.zeros([test_y.shape[0],10])
test_y_onehot[range(test_y.shape[0]),test_y]=1

train_data = list(zip(train_x,train_y_onehot))


# ---------------------------定义模型---------------------------------

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32,name='keep_prob')
x_image = tf.reshape(x,[-1,28,28,1])


#第一次卷积,池化  input=[None,28,28,1] output=[None,14,14,32]
W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1,seed=2018))
b_conv1 = tf.Variable(tf.constant(0.1,shape=[32]))
conv2d_1 = tf.nn.conv2d(x_image,W_conv1,strides=[1,1,1,1],padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(conv2d_1)
#dropout
keep_prob=tf.placeholder("float")
h_fc1_drop=tf.nn.dropout(h_conv1,keep_prob)
h_pool1 = tf.nn.max_pool(h_fc1_drop,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')



# 全连接层1 input=[None,14,14,32] output=[None, 10]
h_pool2_flat = tf.reshape(h_pool1,[-1,14*14*32])


W_fc2 = tf.Variable(tf.truncated_normal([14*14*32,10],stddev=0.1,seed=2018))
b_fc2 = tf.Variable(tf.constant(0.1,shape=[10]))
wx_plus_b2 = tf.matmul(h_pool2_flat,W_fc2) + b_fc2
prediction = tf.nn.softmax(wx_plus_b2)


#交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
#训练
# train_step = tf.train.ProximalAdagradOptimizer(0.1).minimize(loss)
lr = tf.Variable(0.001, dtype=tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# ---------------------------加载数据---------------------------------

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置

#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(50):
        train_batches = batch_iter(train_data)
        for itr,batch in enumerate(train_batches):
            x_batch, y_batch = zip(*batch)
            x_batch, y_batch = np.array(x_batch),np.array(y_batch)
            _, acc, train_loss = sess.run([train_step,accuracy,loss],feed_dict={x:x_batch,y:y_batch,keep_prob:1.0})
        val_acc = sess.run(accuracy,feed_dict={x:test_x,y:test_y_onehot,keep_prob:1.0})
        print("Epoch: {}, Loss: {:.4}, Train acc: {:.4}, Test acc: {:.4}".format(epoch,train_loss,acc, val_acc))
        
    
    


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch: 0, Loss: 1.605, Train acc: 0.862, Test acc: 0.8397
